##  <b>Исследование надёжности заёмщиков:</b> 
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

    
###  Часть 1. Изучение общей информации

    
###  Часть 2. Предобработка данных
    
2.1. Нахождение и ликвидация пропусков

2.2. Замена типа данных

2.3. Обработка дубликатов

2.4. Лемматизация

2.5. Категоризация
    
###  Часть 3. Ответы на вопросы
    
3.1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

3.2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

3.3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

3.4. Как разные цели кредита влияют на его возврат в срок?
    
### Часть 4. Общий вывод

### Шаг 1. Откройте файл с данными и изучите общую информацию. 
<a id='section1'></a>


In [4]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')

display(df.info())
display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Всего в таблице 12 столбцов, 2 столбца принадлежат типу float, 5 - int, 5 - object.
Каждая строка таблицы содержит информацию о клиентах банка. Названия столбцов корректны. 
В ячейках сразу обнаруживаются некорректные данные: отрицательные значение в столбце days_employed, разные регистры строк столбца education. Но для исследования зависимости возврата кредита в срок от информации о наличии детей, семейного положения, уровня дохода и цели кредита клиента данные из этих столбцов не потребуются, поэтому на дальнейшую обработку пройдут столбцы children, family_status, family_status_id, dept, total_income, purpose. 


### Шаг 2. Предобработка данных
<a id='section2'></a>

### Обработка пропусков
<a id='section3'></a>

In [2]:
display(df.isnull().sum()) #суммируем пропущенные значения в данных
df.loc[df['total_income'].isna(), 'total_income'] = df['total_income'].median() #заполняем пропущенные значения медианой
display(df.isnull().sum()) #проверка


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

### Вывод

На этапе предобработки в данных обнаружились пропуски в столбца total_income и days_employed.
Значения в total_income - количественные и отражают уровень заработной платы. Это важный фактор для получения кредита, по всей вероятности данные могли потеряться при выгрузке, а не пропущены при сборе данных.
Т.к. в подобных данных зачастую встречаются выбросы, которые могут повлиять на среднее значение, заменим пропуски не им, а медианой.
Столбец days_employed не потребуется в дальнейшем анализе, поэтому его не обрабатываем. 

### Замена типа данных
<a id='section4'></a>

In [3]:
df['total_income'] = df['total_income'].astype('int')


### Вывод

Данные столбца total_income имеют тип float. Для анализа заработной планы удобнее будет работать с целочисленными переменными, значения после точки не должны оказать значительного влияния на результаты, поэтому переведём тип методом astype.

### Обработка дубликатов
<a id='section5'></a>

In [4]:

df['education'] = df['education'].str.lower() #приводим данные в столбце education к нижнему регистру

display('Количество дубликатов до:', df.duplicated().sum())
df[df.duplicated(keep = False)].sort_values(by = df.columns.values.tolist())
df = df.drop_duplicates().reset_index(drop = True) #удаляем дубликаты, корректируя индексы столбцов
display('Количество дубликатов после:', df.duplicated().sum())


'Количество дубликатов до:'

71

'Количество дубликатов после:'

0

### Вывод

Применяя метод duplicated() обнаруживаем полные дубликаты в таблице(их меньше 1%, возможно, некоторые строки были повторно занесены в таблицу по случайности), удаляем их методом drop_duplicates() и проверяем результат.

### Лемматизация
<a id='section6'></a>

In [5]:
from pymystem3 import Mystem 
m = Mystem()

def word_lemmatize(text): #вводим функцию лемматизации
    for i in text:
        lem_text = m.lemmatize(text)
        return lem_text
df['purpoze_lemmatize'] = df['purpose'].apply(word_lemmatize) #применяем функцию к столбцу и создаём новый столбец


from collections import Counter #подсчитываем число упоминаний лемм в тексте
display(Counter(df['purpoze_lemmatize'].sum())) 


Counter({'покупка': 5897,
         ' ': 33570,
         'жилье': 4460,
         '\n': 21454,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

### Вывод

С помощью библиотеки pymystem3 в элементах столбца с целями кредита были выделены леммы и добавлены в новый столбец датасета, который будет использован в последующей работе над категоризацией. Также с помощью счётчика подсчитаем встречаемость лемм. 

### Категоризация данных
<a id='section7'></a>

In [6]:
income_median = df['total_income'].median()
def total_income_group(row):
    total_income = row['total_income']
    
    if total_income <= income_median/2:
        return 'ниже среднего'
    if total_income < income_median + income_median/2:
        return 'средний'
    return 'выше среднего'
df['total_income_group'] = df.apply(total_income_group, axis = 1)


def children_amount_group(row):
    children_amount = row['children']
    
    if children_amount < 1:
        return 'бездетный(ая)'
    if children_amount <= 2:
        return '1-2 ребенка'
    if children_amount >=3:
        return 'многодетный(ая)'
    
df['children_amount'] = df.apply(children_amount_group, axis = 1)


def purpose_group(row):
    purpose = row['purpoze_lemmatize']
    
    if 'автомобиль' in purpose:
        return 'автокредит'
    if 'свадьба' in purpose:
        return 'кредит на свадьбу'
    if 'недвижимость' in purpose or 'жилье' in purpose and 'ремонт' not in purpose:
        return 'кредит на покупку недвижимости'
    if 'образование' in purpose:
        return 'кредит на получение образования'
    if 'ремонт' in purpose:
        return 'кредит на ремонт'
    
df['purpose_group'] = df.apply(purpose_group, axis = 1)

def age_group(row):
    age = row['dob_years']
    if age < 45:
        return 'молодой возраст'
    if age < 60:
        return 'средний возраст'
    return 'пожилой возраст'

df['age_group'] = df.apply(age_group, axis = 1)

df['family_status'] = df['family_status'].str.lower()
family_status_categories = df.loc[:, 'family_status':'family_status_id']
family_status_categories = family_status_categories.drop_duplicates().reset_index(drop=True)


### Вывод

Данные столбца total_income категоризированы по величине дохода (ниже среднего, средний и выше среднего), выделяю их относительно медианы скорее интуитивно, т.к. надо бы применить методы описательной статистики, которыми пока в достаточной степени не владею, надеюсь, это не фатальная ошибка.
В данных о количестве детей клиента банка были выделены бездетные, многодетные и имеющие 1-2 детей.
Данные о целях кредита категоризованы по леммам, было выделено 5 категорий: автокредит, кредит на свадьбу, кредит на покупку недвижимости, кредит на получение образования, кредит на ремонт.
Также категоризированы данные по возрасту, в соответствии с классификацией ВОЗ и разделены на молодой, средний и пожилой возраст. 
В столбце family_status снижен регистр и выделены категории семейного положения.

### Шаг 3. Ответьте на вопросы
<a id='section8'></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?
<a id='section9'></a>

In [7]:
children_amount_count = df.groupby('children_amount')['debt'].count() #группируем таблицу по кол-ву детей, считаем кол-во заемщиков
children_amount_sum = df.groupby('children_amount')['debt'].sum() #группируем таблицу по кол-ву детей, суммируем число должников
children_amount_proportion = children_amount_sum / children_amount_count #доля должников относительно общего числа заемщиков
print(children_amount_proportion)



children_amount
1-2 ребенка        0.093003
бездетный(ая)      0.075258
многодетный(ая)    0.085526
Name: debt, dtype: float64


### Вывод

Доля задержавших возврат кредита среди клентов банка с 1-2 детьми выше, чем среди бездетных и многодетных. Могу предположить, что это связано с положением молодой семьи, у которой сейчас должно быть относительно много расходов, которые могут повлиять на невозможность оплатить кредит в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?
<a id='section10'></a>

In [8]:
family_status_count = df.groupby('family_status')['debt'].count()
family_status_sum = df.groupby('family_status')['debt'].sum()
family_status_proportion = family_status_sum / family_status_count
display(family_status_proportion)


data_pivot = df.pivot_table(index=['family_status'], columns = 'age_group', values ='debt', aggfunc = ['count'])
display(data_pivot)

family_status
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093471
женат / замужем          0.075452
не женат / не замужем    0.097509
Name: debt, dtype: float64

count                                
age_group             молодой возраст пожилой возраст средний возраст
family_status                                                        
в разводе                         557             155             483
вдовец / вдова                    103             402             454
гражданский брак                 2460             394            1297
женат / замужем                  6774            1309            4256
не женат / не замужем            1914             240             656

### Вывод

Ананилизируем по тому же принципу. Среди должников на первые позиции вышли не женатые/замужние клиенты банка и те, кто состоит в гражданском браке. Возможно люди из данных позиций обладают меньшей социальной ответственностью, нежели люди из других категорий. Также можно предположить, что данные клиенты моложе, это гипотезу проверим в сводной таблице. Действительно, большинство не женатых\замужних и состоящих в гражданском браке относятся к молодому возрасту, поэтому эти факторы могут быть взаимосвязаны. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
<a id='section11'></a>

In [9]:
total_income_count = df.groupby('total_income_group')['debt'].count()
total_income_sum = df.groupby('total_income_group')['debt'].sum()
total_income_proportion = total_income_sum / total_income_count

display(total_income_proportion)

total_income_group
выше среднего    0.071065
ниже среднего    0.069235
средний          0.085067
Name: debt, dtype: float64

### Вывод

Полученные данные свидетельствуют о том, что люди со средним доходом чаще других склонны задерживать оплату по кредиту, но разница между другими категориями незначительна.

- Как разные цели кредита влияют на его возврат в срок?
<a id='section12'></a>

In [10]:
purpose_group_count = df.groupby('purpose_group')['debt'].count()
purpose_group_sum = df.groupby('purpose_group')['debt'].sum()
purpose_group_proportion = purpose_group_sum / purpose_group_count
display(purpose_group_proportion)

data_pivot = df.pivot_table(index=['purpose_group'], columns = 'age_group', values ='debt', aggfunc = ['count'])
display(data_pivot)

purpose_group
автокредит                         0.093590
кредит на покупку недвижимости     0.073207
кредит на получение образования    0.092200
кредит на ремонт                   0.057661
кредит на свадьбу                  0.080034
Name: debt, dtype: float64

count                  \
age_group                       молодой возраст пожилой возраст   
purpose_group                                                     
автокредит                                 2352             523   
кредит на покупку недвижимости             5611            1161   
кредит на получение образования            2207             479   
кредит на ремонт                            351              70   
кредит на свадьбу                          1287             267   

                                                 
age_group                       средний возраст  
purpose_group                                    
автокредит                                 1431  
кредит на покупку недвижимости             3432  
кредит на получение образования            1327  
кредит на ремонт                            186  
кредит на свадьбу                           770

### Вывод

Чаще всего оплату задерживают люди, взявшие кредит на автомобиль и образование. Также связываем эти данные с возрастом клиентов банка и наблюдаем, что большая часть клиентов принадлежит группе молодых. Данна категория людей может обладать более низкой финансовой грамотностью, в силу опыта, поэтому, возможно, они и задерживают выплату по кредиту.
Самый низкий процент наблюдается среди задолжности на ремонт, это может быть связано с меньшей суммой займа в сравнении с другими категориями.

### Шаг 4. Общий вывод
<a id='section13'></a>

Большая часть клиентов банка, задержавших оплату по кредиту, состоят в гражданском браке и/или не женаты(замужем), имеют 1-2 детей, имеют средний доход и берут кредит на автомобиль или получение образования. Большинство людей из данных категорий также относятся к молодой возрастной группе, в которой может наблюдатся более низкая финансовая грамотность и ответственность.
Самыми надежными клиентами являются люди, которые принадлежат категориям бездетные, потерявшие супруга (вдовцы/вдовы), имеющие доход ниже среднего и взявшие кредит на ремонт. 